# My first lab

In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [3]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

In [6]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [7]:
system_prompt = "You are an assistant that analyzes the contents of a post on LinkedIn and provides a short, relevant, and thoughtful comment on it. \
Ignore text that might be navigation related. Keep the tone professional and conversational. Respond in plain text."


In [8]:
def user_prompt_for(website):
    return f"""
You are reading a LinkedIn post titled: "{website.title}"

Here is the full content of the post:
{website.text}

Write a short, thoughtful comment that could be posted as a reply.
"""


In [9]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [10]:
# And now: call the OpenAI API. You will get very familiar with this!

def generate_comment(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [11]:
generate_comment("https://www.linkedin.com/posts/roman-sheremeta-14972a50_this-just-keeps-getting-better-now-elon-activity-7310735395833405442-sf2M?utm_source=share&utm_medium=member_desktop&rcm=ACoAAETGRocBBwfTpcFHYHGUMNKtk0S5TVTccRw")

"It's certainly concerning when private individuals are involved in sensitive government matters, especially when it comes to national security. It highlights the need for clarity about roles and responsibilities within our systems. Collaboration is crucial, but it must be balanced with the seriousness and gravity of the issues at hand. Thank you for bringing this to our attention, Roman."

In [12]:
# A function to display this nicely in the Jupyter output, using markdown

def display_comment(url):
    comment = generate_comment(url)
    display(Markdown(comment))

In [13]:
display_comment("https://www.linkedin.com/posts/roman-sheremeta-14972a50_this-just-keeps-getting-better-now-elon-activity-7310735395833405442-sf2M?utm_source=share&utm_medium=member_desktop&rcm=ACoAAETGRocBBwfTpcFHYHGUMNKtk0S5TVTccRw")

This situation certainly raises important questions about security and information sharing in sensitive contexts. Miscommunication can have significant consequences, especially when it involves national security matters. It will be interesting to see how this unfolds and what measures may be put in place to prevent similar incidents in the future.